In [1]:
import os 
import pandas as pd
import numpy as np
import json 

import open3d as o3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, Sum, CompoundKernel
# import plotly.graph_objects as go

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
def down_sample(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    downpcd = pcd.voxel_down_sample(voxel_size=0.005)
    return np.asarray(downpcd.points)

In [37]:
# list files 
dirpath = os.getcwd()
pcd_path = os.path.join(dirpath, "point_clouds")
udder_path = os.path.join(pcd_path, "udder")
kp_path = os.path.join(pcd_path, "keypoints")
quarter_path = os.path.join(pcd_path, "quarters")

out_path = os.path.join(dirpath, "features_dict")

filenames = [file.replace(".json", "") for file in os.listdir(kp_path)]
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-5, 1e2))  + WhiteKernel()

In [52]:
skip_files = []
cnt = 0
for file in filenames:
    print(cnt)
    cnt+=1
    udder_pc = np.load(os.path.join(udder_path, file + ".npy"))
    
    with open(os.path.join(quarter_path, file + ".json")) as f:
        quarter_dict = json.load(f)
    
    with open(os.path.join(kp_path, file + ".json")) as f:
        kp_dict = json.load(f)
        
    pcd_dict = {}
    teat_dict = {} 
    skip = 0
    for key in quarter_dict.keys():
        quarter = np.array(quarter_dict[key])
        teat = np.array(kp_dict[key]['xyz_tf'])
        dist_teat = np.array([np.linalg.norm(teat[:2]-point[:2]) for point in quarter])
        max_dist_quarter = np.max(dist_teat)
        
        delta_z = []
        radi = []
        z_val = []
        rad = 0.001
        last_rad = 0
        step = 0.002
        last_z = teat[2]
        
        while rad < max_dist_quarter:
            # sort the distances
            # points within 1mm 
            condition = [(d <= rad) & (d > last_rad) for d in dist_teat]
            if sum(condition) >0:
                circle_idx = np.where(condition)[0]
                cirle_dist = dist_teat[circle_idx]
                circle_pts = quarter[circle_idx]
                circle_z = np.min(circle_pts[:, 2])
                
                # max distance in circle
                max_dist_circle = np.max(cirle_dist)
                
                # look at the z change
                dif = abs(circle_z - last_z)
                delta_z.append(dif)
                radi.append(rad)
                z_val.append(circle_z)
                
                # update stuff
                last_rad = rad   
                last_z = circle_z
            rad += step 
        delta_z = np.array(delta_z)
        radi = np.array(radi)
        z_val = np.array(z_val)
        
        small_gradient = [(d >= 0) & (d < 0.001) for d in delta_z]
        small_idx = np.where(small_gradient)[0]
        candidates = np.array(radi[small_idx])
        candidates_h = np.array(z_val[small_idx])
        
        try:
            cand_idx = np.where((candidates[1:] - candidates[:-1]) > step*2)[0][0]
            teat_radius = candidates[cand_idx+1] -step
            teat_z = candidates_h[cand_idx+1]
        except:
            skip = 1
            skip_files.append(file)
    
        if skip == 0:
            # flag the points that are teat 
            base_idx = np.where(dist_teat < teat_radius)[0]
            lowpts_idx = np.where(quarter[:, 2] < teat_z)[0]
            base_coords = np.row_stack((quarter[base_idx], quarter[lowpts_idx]))
            quarter2 = quarter.copy()
            quarter2[base_idx, 2] = np.nan
            quarter2[lowpts_idx, 2] = np.nan
            
            dspc = down_sample(quarter2)
            
            missing = dspc[np.isnan(dspc[:, 2]), :2]
            observed = dspc[~np.isnan(dspc[:, 2]), :]
            
            gaussian_process = GaussianProcessRegressor(kernel=kernel)
            gaussian_process.fit(observed[:,:2], observed[:,2])
            vals = gaussian_process.predict(missing[:, :2], return_std=False)
            predicted = np.column_stack([missing, np.transpose(vals)])
            pcd_dict[key] = {"teat_pts": base_coords.tolist(), "obs_pts": observed.tolist(), "pred_pts": predicted.tolist()}
            
            leng_all = np.array([np.linalg.norm(teat - point) for point in predicted])
            teat_base = predicted[np.argsort(leng_all)[0]]
            teat_length = leng_all[np.argsort(leng_all)[0]]
            
            teat_dict[key] = {"bottom": teat.tolist(), "tip": teat_base.tolist(), "length": teat_length}
        
    with open(os.path.join(pcd_path, "teat", file + ".json"), 'w') as f:
        json.dump(pcd_dict, f)
    with open(os.path.join(out_path, "teat_length", file + ".json"), 'w') as f:
        json.dump(teat_dict, f)

print(skip_files)

C:\Users\marie\miniconda3\envs\napari-env2\lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

C:\Users\marie\miniconda3\envs\napari-env2\lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

C:\Users\marie\miniconda3\envs\napari-env2\lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

C:\Users\marie\miniconda3\envs\napari-env2\lib\site-packages\sklearn\gaussian_proc

In [50]:
teat_dict

{'lf': {'bottom': array([0.03796229, 0.07697083, 0.06092589]),
  'tip': array([0.02624866, 0.08415521, 0.08539687]),
  'length': 0.02806516226785275}}

In [43]:
def plot_stuff(observed, predicted, base_coords):
    points = observed
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    fig =  go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color="slateblue", size = 1, opacity = 0.5), name = "Udder")])
    
    points = predicted 
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color="blue", size = 2), name = "ols"))
    
    points = base_coords
    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(color="yellow", size = 2), name = "teat"))
    return fig